# Homework 5 - Retrieval Models: Query Likelihood
CS 437  
Fall 2025  
Dr. Henderson  
_v1_

---

**First copy your `hw4lib.py` from the previous assignment as `hw5lib.py` in the same directory as this notebook**

These imports will be useful for the following exercises. You may add other imports here if desired.

In [1]:
import importlib
import os

import cacmlib
import hw5lib as hw5

---

You will be doing most of your development in the `hw5lib.py` module and then running unit tests and other code from this notebook. The `importlib` module has been included above so that you can reload your `hw5lib.py` changes without having to restart the kernel. This makes iterating much faster but can sometimes lead to confusing states of the notebook (if you don't understand Python imports). If you get stuck on a bug it's a good idea to reset the kernel to rule out any reloading issues.

_Run this cell anytime you make changes to `hw5lib.py` to update the current state of your notebook_

In [19]:
importlib.reload(hw5)

<module 'hw5lib' from '/home/ehenders/VOL/CLASSES/CS437/homework/source/hw5/hw5lib.py'>

## 1. Vector Space Model

The first model we studied is based on representing documents and queries as weighted vectors of terms and ranking documents based on a similarity measure (e.g. cosine distance). For this assignment, you will build on your previous work to build a a vector space ranking model.

Run the cell below to initiate your database and add the documents in the `tests` subdirectory.

In [5]:
hw5.create_db()
assert os.path.isfile('index.db')

hw5.add_docs('tests')

The equation derived in the book that we will use for document vector term weights is:  

#### $ w_{ik} = \frac{(\log(f_{ik}) + 1) \cdot \log(N/n_k) }{ \sqrt{ \sum\limits_k \left[ (\log(f_{ik}) + 1) \cdot \log(N/n_k)\right]^2 } } $

$ f_{ik} $ is the frequency of the term $ k $ in the documentd $ d_i $  
$ N $ is the number of documents  
$ n_k $ is the number of documents the term $ k $ appears in



1.1 (55 pts) Create a function named `index_vector_model()` in the `hw5lib.py` file that takes no parameters. Your function should read the current list of documents in the `index.db` database and for each document it should compute the vector weights for each term (use standard preprocssing to filter terms) and store them in a new table in the database.

In [6]:
hw5.index_vector_model()

1.2 (15 pts) Create a function named `get_document_vector()` in the `hw5lib.py` file that takes a single string filename and returns the associated weighted terms of the document vector as a sorted list (in decreasing weight).

In [9]:
results = hw5.get_document_vector('tests/doc2.txt')[:5]
assert results == [('queri', 0.369569331014455), ('databas', 0.320178350159223), ('store', 0.21827360034479323), ('retriev', 0.21827360034479323), ('sql', 0.21827360034479323)], results
results = hw5.get_document_vector('tests/doc7.txt')[:5]
assert results  == [('analyt', 0.394498512110325), ('extract', 0.23299717628792277), ('insight', 0.23299717628792277), ('statist', 0.23299717628792277), ('analysi', 0.23299717628792277)], results

To rank documents requires computing a document vector for the query and then using the cosine similarity measure to produce a similarity score that can be used for ranking. The term weight equation is the same as above, except the terms to use, i.e. $ f_{ik} $, are taken from the query, not the document. To compute the cosine similarity, simply take the dot product of the query vector and the document vector.

1.3 (25 pts) Create a function named `rank_document_model()` in the `hw5lib.py` file that takes a query string and returns a ranked list of relevant documents as a tuple `(score, filename)`. Your function should compute the term weights of the query vector and then use the cosine similarity measure to rank all the documents in the database.

In [14]:
results = hw5.rank_document_model("machine learning")
assert results == [(0.25006297536920996, 'tests/doc9.txt'), (0.24903727223083577, 'tests/doc1.txt'), (0.07552280941097185, 'tests/doc5.txt')], results
results = hw5.rank_document_model("database transactions")
assert results == [(0.3807430255445131, 'tests/doc2.txt'), (0.10723037781462168, 'tests/doc3.txt')], results

1.4 (5 pts) Create a function named `query_document_model()` which takes a query string and returns a ranked list of relevant documents as a list of filename strings. This is just a wrapper around the previous function so it can be used in the evaluation.

In [16]:
results = hw5.query_document_model("machine learning")
assert results == ['tests/doc9.txt', 'tests/doc1.txt', 'tests/doc5.txt'], results
results = hw5.query_document_model("database transactions")
assert results == ['tests/doc2.txt', 'tests/doc3.txt'], results

# 2. Query Likelihood Model

We looked at several algorithms based on language models. For this assignment you will implement the _Query Likelihood Model_, which computes the probability of the query $ Q $ being generated from the language model of the document $ D $ using the equation:  
#### $ \log P(Q|D) = \sum\limits_n \log p(q_n|D) $  

### $ p(q_n|D) = \frac{f_{q_n,D} + \mu \frac{c_{q_n}}{|C|}}{|D| + \mu} $  

$n$ is used to index each of the query terms, and $ c_{q_n} $ is the count of query term $q_n$ in the entire document collection. $|C|$ is the count of all term occurrences in the collection and $ |D| $ is the count of all term occurrences in the document $ D $

2.1 (55 pts) Create a function named `index_query_likelihood()` in the `hw5lib.py` file that takes no parameters. Your function should read the current list of documents in the `index.db` database and for each document it should estimate the log probability $ p(q_n|D) $ for each term and store it in a new table in the database.

In [18]:
hw5.index_query_likelihood()

2.2 (15 pts) Create a function named `get_document_likelihood()` in the `hw5lib.py` file that takes a single string filename and returns the associated term likelihoods of the document as a sorted list (in decreasing likelihood).

In [21]:
results = hw5.get_document_likelihood('tests/doc2.txt')[:5]
assert results == [('data', 0.002159217414503846), ('system', 0.001956289443551291), ('process', 0.0018175973156571504), ('databas', 0.0016591423226115568), ('improv', 0.0016418316588775997)], results
results = hw5.get_document_likelihood('tests/doc7.txt')[:5]
assert results  ==  [('data', 0.002172469687489766), ('system', 0.001957254083316356), ('process', 0.0018306811369178007), ('larg', 0.0014888925863458355), ('platform', 0.0014888925863458355)], results

To rank documents requires computing a score for each document by summing all the log probabilities for terms in the document that appear in the query.

2.3 (25 pts) Create a function named `rank_query_likelihood()` in the `hw5lib.py` file that takes a query string and returns a ranked list of relevant documents as a tuple `(score, filename)`. Your function should compute the query likelihood of each document and use it to rank all the documents in the database.  
_IMPORTANT: Try to optimize your database calls or the evaluation in section 3 below could take a very long time._

In [24]:
results = hw5.rank_query_likelihood("machine learning")
assert results == [(0.003125369390242128, 'tests/doc1.txt'), (0.003123832071309397, 'tests/doc9.txt'), (0.001485961695427832, 'tests/doc5.txt')], results
results = hw5.rank_query_likelihood("database transactions")
assert results == [(0.0026517676772958424, 'tests/doc2.txt'), (0.0016230908384346938, 'tests/doc3.txt')], results

2.4 (5 pts) Create a function named `query_likelihood()` which takes a query string and returns a ranked list of relevant documents as a list of filename strings. This is just a wrapper around the previous function so it can be used in the evaluation.

In [26]:
results = hw5.query_likelihood("machine learning")
assert results == ['tests/doc1.txt', 'tests/doc9.txt', 'tests/doc5.txt'], results
results = hw5.query_likelihood("database transactions")
assert results == ['tests/doc2.txt', 'tests/doc3.txt'], results

## 3. Evaluation

We will use the _CACM_ dataset to evaluate your search engine. Indexing the entire _CACM_ dataset can take several minutes. The following cell is commented out so that id does not run during grading -- instead you should run it manually (by uncommenting the code) once you are finished with your implementation. You will include your database as part of your submission.

In [2]:
hw5.create_db()
hw5.add_docs('../../datasets/cacm/docs')

**IMPORTANT**: Please remember to recomment the code in the cell above after you have built your index.

3.1 (10pts) Index the _CACM_ dataset using your vector model.

In [3]:
hw5.index_vector_model()

Run the cell below to get the document vector for `CACM-0002`

In [ ]:
hw5.get_document_vector('../../datasets/cacm/docs/CACM-0002.html')

3.2 (5 pts) Give a brief analysis of the results -- are they expected and what changes if any would you make to your implementation?

3.3 (5 pts) Evaluate your implementation using the cell below.

In [ ]:
importlib.reload(cacmlib)
cacmlib.eval(hw5.query_document_model)

3.4 (10 pts) Index the _CACM_ dataset using your query likelihood model.

In [9]:
hw5.index_query_likelihood()

Run the cell below to get the document likelihood for `CACM-0002`

In [ ]:
hw5.get_document_likelihood('../../datasets/cacm/docs/CACM-0002.html')

3.5 (5 pts) How does it compare to your results from 3.2 and why?

3.6 (5 pts) Evaluate your implementation using the cell below.

In [ ]:
cacmlib.eval(hw5.query_likelihood)

3.7 (5 pts) Compare the results to the vector space model and give possible explanations for the difference.

---

### Submission Instructions

Be sure to ***SAVE YOUR WORK***!  

Next, select Kernel -> Restart Kernel and Run All Cells...

Make sure there are no errors in **Section 1**.

Then uncomment the first cell in **Section 2** and manually run the cells in **Section 2**.

Make sure there are no errors in **Section 2**.

***IMPORTANT***: recomment the code in the first cell in **Section 2** again.

Then submit the following to Canvas:

This notebook (`.ipynb`) file using the original name - do not rename it!

Your `hw5lib.py` file with your implementation.

Your `index.db` file containing the index of the _CACM_ directory.